# Permissions Code Along

### Introduction

Now that we set up a second IAM user, and have access to this user from the command line, let's see what we can do with this user.

### Coding Along

So remember that we are logged in as our `codealong-user`, and that we granted this user both s3 and iam access.  (You can run `python 1-iam.py` to confirm this is still the case).  


Ok, so now let's explore our user's s3 access.

> The code below is in the `2-iam-s3.py` file.

So below, we connect to the s3 resource, and then right away see that we get an access denied error if we try to create an S3 bucket.

```python
s3 = boto3.client('s3')

s3.create_bucket(Bucket = 'jigsawspecialsample') 
# Access denied
```

This makes sense.  Remember that our current user, only has list and read access.  So let's make sure that we still have this ability.  Below, specify a bucket that you have created in your account.

```python
s3.list_objects_v2(Bucket = 'jigsawtxdrinks')
    # Note that we can list ojects from a bucket from the same account, even if that bucket is private
```

```python
s3.put_object(Body='Text Contents', Bucket='jigsawtxdrinks', Key='example.txt')
    # However, for a bucket that is not publicly accessible, we cannot add a new object
```

Ok, so we saw above some expected behavior.  As a user that has s3 get access, we can list objects in a bucket, so long as that bucket is in the same account.  However, we have no ability to create an object in a bucket that is not publicly accessible.

### Interacting with Resource Policies

Ok, now so far we have explored the policies attached to an IAM user.  But now let's see how they interact with the policies on a resource, like S3.

So go to S3 in the AWS web console, and create a new bucket, and after entering a name for the bucket, make it publicly accessible.

> Do this by unchecking the block all public access, which will make the bucket publicly accessible.  And then check the warning checkbox underneath.

<img src="./uncheck-s3.png" width="70%">

Now let's see if this public access enables our `codealong-user` to add an object to the bucket.

```python
s3_client.put_object(Body='Text Contents', Bucket=bucket_name, Key='example.txt')

# An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
```

As we can see, we still get an access denied.

Now let's click on the bucket, and then click on the bucket policy, and add something like the following.

> Note: Under **resource**, you'll need to add the arn (amazon resource name) of the s3 bucket.  And you'll need to add a `/*` at the end to provide access to all of the objects in the bucket.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadPut",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3:PutObject",
                "s3:PutObjectAcl",
                "s3:GetObject",
                "s3:GetObjectAcl",
                "s3:DeleteObject"
            ],
            "Resource": "arn:aws:s3:::codealongbucketjigsaw/*"
        }
    ]
}
```

So let's take a look at this policy above.  Once again, we have outer keys of `Version` and `Statement`. The `Sid` is the statement id, and we are granting an "Allow" effect to various actions. One new key is the `Principal`, which are the users or resources we are granting access to.  

> Above we are specifying a `*` for Principal, which grants this access to any AWS user.  But if you type `aws iam list-users`, you'll see that each user is also granted an arn, and we could paste a list of those arns if we only wanted to grant access to those users.

Under get actions, notice that we are providing the ability to get and put objects, as well as the ability to retreive and change the Acl, which is the access control list.  The access control list defines who has access to an S3 bucket or object (you can read more [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/acl-overview.html)).  

So this time, if we call the `put_object` method, we'll see that we *can* create a new object in the bucket.  Even if our current user does not have that access.

```python
s3.put_object(Body='Text Contents', Bucket=bucket_name, Key='example.txt')
```

If you look at the s3 bucket, you'll see that the object has been added to the bucket.

So here we can see that while there is not explicit put access on the IAM user, because there is explicit access on the bucket itself, that wins out, and we can still create an object.

### Modifying the ACL

Ok, so now let's start modifying the ACL.  We can begin by changing the principal to our a different user.  Type `aws iam list-users`, and copy the arn of a user you want to provide access to (not the codealong-user).  For example, an ARN of one of my users looks like the following:

```json
arn:aws:iam::086729879076:user/total_access
```

And so let's update the policy for the S3 bucket to be something like the following:

```json
"Principal": {
    "AWS": "arn:aws:iam::086729879076:user/total_access"
    },
```

From there, try to call the `put_object` command again, to confirm we no longer have access.

```python
s3.put_object(Body='Text Contents', Bucket=bucket_name, Key='example.txt')
# Access Denied
```

This obviously makes sense as neither our user's IAM policy or the bucket policy is allowing for access.

### Making other updates

What if we would like to make other changes to our bucket.  If you click on the Edit button, under policy, you'll see the option to click on the `Policy generator`.

<img src="./policy-generator.png" width="90%">

So click on that.  And from there, under `Select Type of Policy`, choose the S3 Bucket Policy.

And then fill in some relevant information for the principal, resource, and the amazon resource name (which should be your bucket name).

<img src="./generate-policy.png" width="90%">

Afterwards, if you scroll down, you'll see a generate policy button. And then clicking on that button will display the json for your policy.

### Summary

Ok, so in this lesson, we saw how our user with S3readonlyaccess could in fact read s3 buckets from the account, but could not create new objects or new buckets.  But then we saw that when we explicitly allowed for the access from the bucket, that the user could then create objects in the bucket.

Looking at the ACL, there were new keys of "Principal" where we can specify who the statement applies to.  And we also specified the resource, placing the ARN of the relevant bucket followed by `/*` to allow for access to the objects in the bucket.

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadPut",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3:PutObject",
                "s3:PutObjectAcl",
                "s3:GetObject",
                "s3:GetObjectAcl",
                "s3:DeleteObject"
            ],
            "Resource": "arn:aws:s3:::codealongbucketjigsaw/*"
        }
    ]
}
```

From there, we saw that we can see how to generate other policies through the [AWS policy generator](https://awspolicygen.s3.amazonaws.com/policygen.html).

### Resources

[ACL Overview](https://docs.aws.amazon.com/AmazonS3/latest/userguide/acl-overview.html)

[Iam and bucket policies](https://aws.amazon.com/blogs/security/iam-policies-and-bucket-policies-and-acls-oh-my-controlling-access-to-s3-resources/)

[IAM Policies](https://start.jcolemorrison.com/aws-iam-policies-in-a-nutshell/)